In [4]:
import pandas as pd
import re
import plotly.express as px

In [5]:
def eliminar_urls(texto: str) -> str:
    """Elimina URLs de un texto dado."""
    if pd.isnull(texto):
        return texto
    return re.sub(r'https?://\S+|www\.\S+', '', texto)


def clasificar_edad_rango(edad) -> str | None:
    """Clasifica una edad numérica en rangos predefinidos."""
    if pd.isna(edad):
        return None
    edad = int(edad)
    if edad < 30:
        return "<30"
    elif edad < 40:
        return "30-39"
    elif edad < 50:
        return "40-49"
    elif edad < 60:
        return "50-59"
    elif edad < 70:
        return "60-69"
    return "70+"


def obtener_quinquenios(legislaturas_str: str) -> str:
    """
    Convierte una cadena de años separados por comas en rangos quinquenales.
    Ejemplo: "2000, 2003" → "2000-2004, 2000-2004"
    """
    años = {int(x.strip()) for x in legislaturas_str.split(",") if x.strip().isdigit()}
    quinquenios = {
        f"{1975 + 5 * ((año - 1975) // 5)}-{1975 + 5 * ((año - 1975) // 5) + 4}"
        for año in años
    }
    return ", ".join(sorted(quinquenios))


def agrupar_seguidores(seguidores: int) -> str:
    """Agrupa cuentas según cantidad de seguidores."""
    if seguidores < 1_000:
        return "Menos de 1k"
    elif seguidores < 10_000:
        return "1k - 10k"
    elif seguidores < 100_000:
        return "10k - 100k"
    elif seguidores < 1_000_000:
        return "100k - 1M"
    return "Más de 1M"


def agrupar_posts(posts: int) -> str:
    """Agrupa cuentas según cantidad de publicaciones."""
    if posts < 1_000:
        return "Menos de 1k"
    elif posts < 10_000:
        return "1k - 10k"
    elif posts < 50_000:
        return "10k - 50k"
    elif posts < 100_000:
        return "50k - 100k"
    return "Más de 100k"


def agrupar_fechas(fechas: int) -> str:
    """Agrupa fechas en rangos anuales relevantes."""
    if fechas < 2010:
        return "Antes de 2010"
    elif fechas < 2014:
        return "2010 - 2014"
    elif fechas < 2020:
        return "2014 - 2019"
    elif fechas < 2025:
        return "2020 - 2024"
    return "2025 y después"

In [ ]:
ruta_excel = "clasificador_analisis/analisis/datasets/politicos_etiquetado_completo.xlsx"


df_metadata = pd.read_excel(ruta_excel, sheet_name="Metadata")
df_posts = pd.read_excel(ruta_excel, sheet_name="Posts")
df_comentarios = pd.read_excel(ruta_excel, sheet_name="Comentarios")

print("\n📄 HOJA: Posts")

columnas_a_mostrar = ["Tono", "Tema"]
for columna in columnas_a_mostrar:
    if columna in df_posts.columns:
        print(f"\n📌 Distribución de {columna}:")
        print(df_posts[columna].value_counts(dropna=False))


📄 HOJA: Posts

📌 Distribución de Tono:
Positivo    5643
Negativo    3025
Neutro       851
Name: Tono, dtype: int64

📌 Distribución de Tema:
Gestión Pública e Instituciones                 4415
Economía, Empresa, Empleo e Infraestructuras    2453
Sociedad, Igualdad y Derechos                   1790
Otros                                            861
Name: Tema, dtype: int64


In [7]:
print("\n📄 HOJA: Comentarios")

columnas_a_mostrar = ["Tono", "Tono_Respuesta"]
for columna in columnas_a_mostrar:
    if columna in df_comentarios.columns:
        print(f"\n📌 Distribución de {columna}:")
        print(df_comentarios[columna].value_counts(dropna=False))


📄 HOJA: Comentarios

📌 Distribución de Tono:
Negativo    21649
Neutro       5371
Positivo     2946
Name: Tono, dtype: int64

📌 Distribución de Tono_Respuesta:
NaN         29766
Negativo       81
Neutro         60
Positivo       59
Name: Tono_Respuesta, dtype: int64


In [8]:
columnas_metadata = ["Descripción"]
columnas_posts = ["Contenido", "Contenido_Traducido"]
columnas_comentarios = ["Contenido", "Comentario_Traducido", "Respuesta", "Respuesta_Traducida"]

df_metadata[columnas_metadata] = df_metadata[columnas_metadata].applymap(eliminar_urls)
df_posts[columnas_posts] = df_posts[columnas_posts].applymap(eliminar_urls)
df_comentarios[columnas_comentarios] = df_comentarios[columnas_comentarios].applymap(eliminar_urls)

## Distribución posts y comentarios por mes

In [9]:
df_posts = pd.read_excel(ruta_excel, sheet_name="Posts")
df_comentarios = pd.read_excel(ruta_excel, sheet_name="Comentarios")

for df in [df_posts, df_comentarios]:
    df["Fecha_Publicación"] = pd.to_datetime(df["Fecha_Publicación"], errors="coerce")

posts_por_fecha = df_posts["Fecha_Publicación"].dt.date.value_counts().sort_index()
comentarios_por_fecha = df_comentarios["Fecha_Publicación"].dt.date.value_counts().sort_index()

df_plot = pd.DataFrame({
    "Fecha": list(posts_por_fecha.index) + list(comentarios_por_fecha.index),
    "Cantidad": list(posts_por_fecha.values) + list(comentarios_por_fecha.values),
    "Tipo": ["Posts"] * len(posts_por_fecha) + ["Comentarios"] * len(comentarios_por_fecha)
})

fig = px.line(
    df_plot,
    x="Fecha",
    y="Cantidad",
    color="Tipo",
    title="Distribución temporal de publicaciones y comentarios",
    labels={"Cantidad": "Número de mensajes", "Fecha": "Fecha"}
)

fig.update_traces(mode="lines+markers")
fig.update_layout(
    hovermode="x unified",
    width=1000,
    height=500,
    margin=dict(l=20, r=20, t=60, b=20),
    showlegend=True
)

fig.show()

### Distribución de Cargo

In [10]:
fig_cargo = px.pie(
    df_metadata,
    names="Cargo",
    title="Distribución de Cargo"
)

fig_cargo.update_traces(
    textposition="inside",
    textinfo="percent+label",
    pull=[0] * df_metadata["Cargo"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_cargo.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=20),
    showlegend=True
)

fig_cargo.show()

### Distribución de Género

In [11]:
fig_genero = px.pie(
    df_metadata,
    names="Género",
    title="Distribución de Género en Metadata"
)

fig_genero.update_traces(
    textposition="inside",
    textinfo="percent+label",
    pull=[0] * df_metadata["Género"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_genero.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=20),
    showlegend=True
)

fig_genero.show()

### Distribución de Estudios

In [12]:
fig_estudios = px.pie(
    df_metadata,
    names="Estudios",
    title="Distribución de Estudios en Metadata"
)

fig_estudios.update_traces(
    textposition="inside",
    textinfo="percent+label",
    pull=[0] * df_metadata["Estudios"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_estudios.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=20),
    showlegend=True
)

fig_estudios.show()

### Distribución de Campos de Estudio

In [13]:
fig_campo = px.pie(
    df_metadata,
    names="Campo",
    title="Distribución de campo de estudio en Metadata"
)

fig_campo.update_traces(
    textposition="inside",
    textinfo="percent+label",
    pull=[0] * df_metadata["Campo"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_campo.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=10),
    showlegend=True
)

fig_campo.show()

### Distribución de Comunidades Autónomas

In [14]:
fig_comunidad = px.pie(
    df_metadata,
    names="Comunidad Autónoma",
    title="Distribución de comunidades en Metadata"
)

fig_comunidad.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Comunidad Autónoma"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_comunidad.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=5),
    showlegend=False
)

fig_comunidad.show()

### Distribución de Partidos Políticos

In [15]:
colores_partidos = {
    "PSOE": "#ff0000",
    "PP": "#189ad3",
    "SUMAR": "#ff0065",
    "VOX": "#74d600",
    "JxCAT-JUNTS": "#43e8d8",
    "EAJ-PNV": "#389844",
    "ERC": "#fdb73e",
    "EH Bildu": "#3fa0a3",
    "CCa": "#fffff2",
    "PRC": "#d6ff00",
    "BNG": "#b0cfff",
    "Más Madrid": "#52eb86",
    "UPN": "#0059b3",
}

fig_partido = px.pie(
    df_metadata,
    names="Partido",
    title="Distribución de partido en Metadata",
    color="Partido",
    color_discrete_map=colores_partidos
)

fig_partido.update_traces(
    textposition="inside",
    textinfo="percent+label",
    pull=[0] * df_metadata["Partido"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_partido.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=10),
    showlegend=True
)

fig_partido.show()

### Distribución por Legislaturas

In [16]:
df_metadata["Número de Legislaturas"] = df_metadata["Número de Legislaturas"].apply(
    lambda x: "Más de 3" if x > 3 else str(x)
)

In [17]:
fig_legislaturas = px.pie(
    df_metadata,
    names="Número de Legislaturas",
    title="Distribución de nº de legislaturas"
)

fig_legislaturas.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Número de Legislaturas"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_legislaturas.update_layout(
    width=800,
    height=700,
    margin=dict(l=20, r=20, t=60, b=5),
    showlegend=False
)

fig_legislaturas.show()

### Distribución de Edad

In [18]:
df_metadata["Rango_Edad"] = df_metadata["Edad"].apply(clasificar_edad_rango)

In [19]:
fig_edad_rangos = px.pie(
    df_metadata,
    names="Rango_Edad",
    title="Distribución de edad"
)

fig_edad_rangos.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Rango_Edad"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_edad_rangos.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=5),
    showlegend=False
)

fig_edad_rangos.show()

### Distribución por rango de Legislaturas

In [20]:
df_metadata["Legislaturas"] = df_metadata["Legislaturas"].astype(str)

df_metadata["Rango_Legislaturas"] = df_metadata["Legislaturas"].apply(obtener_quinquenios)

In [21]:
frecuencia_quinquenios = (
    df_metadata["Rango_Legislaturas"]
    .dropna()
    .str.split(r",\s*")
    .explode()
    .str.strip()
    .value_counts()
    .sort_index()
)

fig = px.bar(
    x=frecuencia_quinquenios.index,
    y=frecuencia_quinquenios.values,
    title="Distribución de Legislaturas por Rango",
    labels={"x": "Rango de Años", "y": "Número de personas"}
)

fig.update_layout(
    width=800,
    height=500,
    bargap=0.2,
    margin=dict(l=20, r=20, t=60, b=60)
)

fig.show()

### Distribución por rango de Seguidores y Posts

In [22]:

df_metadata["Rango_Seguidores"] = df_metadata["Seguidores"].apply(agrupar_seguidores)

df_metadata["Rango_Posts"] = df_metadata["Posts"].apply(agrupar_posts)

In [23]:
fig_rango_seguidores = px.pie(
    df_metadata,
    names="Rango_Seguidores",
    title="Distribución Rango de Seguidores"
)

fig_rango_seguidores.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Rango_Seguidores"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_rango_seguidores.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=10),
    showlegend=False
)

fig_rango_seguidores.show()

In [24]:
fig_rango_posts = px.pie(
    df_metadata,
    names="Rango_Posts",
    title="Distribución Rango de Posts"
)

fig_rango_posts.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Rango_Posts"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_rango_posts.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=10),
    showlegend=False
)

fig_rango_posts.show()

### Distribución por rango de creación de cuenta

In [25]:
df_metadata["Comienzo en X/Twitter rango"] = df_metadata["Comienzo en X/Twitter"].apply(agrupar_fechas)

In [26]:
fig_inicio_twitter = px.pie(
    df_metadata,
    names="Comienzo en X/Twitter rango",
    title="Distribución por año de inicio en X/Twitter"
)

fig_inicio_twitter.update_traces(
    textposition="inside",
    textinfo="percent+label+value",
    pull=[0] * df_metadata["Comienzo en X/Twitter rango"].nunique(),
    marker=dict(line=dict(width=0))
)

fig_inicio_twitter.update_layout(
    width=800,
    height=800,
    margin=dict(l=20, r=20, t=60, b=10),
    showlegend=False
)

fig_inicio_twitter.show()

In [ ]:
ruta_salida = "clasificador_analisis/analisis/datasets/politicos_etiquetado_actualizado.xlsx"

with pd.ExcelWriter(ruta_salida, engine="openpyxl") as writer:
    df_metadata.to_excel(writer, sheet_name="Metadata", index=False)
    df_posts.to_excel(writer, sheet_name="Posts", index=False)
    df_comentarios.to_excel(writer, sheet_name="Comentarios", index=False)